In [1]:
import snowflake.connector
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
import pandas as pd
import numpy as np
import copy

In [2]:
# Snowflake connection - DSC
with open('/etc/security/snowflake/' + "/rsa_plbiap01dy.p8", "rb") as key:
    p_key = serialization.load_pem_private_key(
        key.read(),
        password='snowflake'.encode(),
        backend=default_backend())

pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
        user='plbiap01dy',
        account='hfsg_prod.us-east-1.privatelink',
        private_key=pkb,
        warehouse='DSC_PLBI_PRD_MFG_WHS',
        role='plbiap01dy_prd_pii_role'
)

# Snowflake queries
query_scoremart = """
SELECT ulm.*, 
aq.POL_ID,
ap.TERM_LATST_UBI_SCORE,
ap.TLMTC_POL_BUCKET_CD
FROM DSC_PLDS_DB.APP_AUTOMATA_PRD.PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY ulm
JOIN PRD_PL_DB.APP_DCPA_DM.AUTO_QUOTE_LATST_VW aq
USING (QCN)
JOIN DSC_PLBI_DB.APP_AUTO_PRD.AUTO_PERSISTENCY_NEWCO ap
ON aq.POL_ID = ap.POL_ID
WHERE ulm.trans_dt BETWEEN '2022-01-01' AND '2024-09-30' 
AND issue_ind = 'Y' 
AND cf_veh_telematic_ind = 'Y'
AND ap.POL_EFF_DT <= '2024-09-30'
QUALIFY ROW_NUMBER() OVER (PARTITION BY ap.POL_ID ORDER BY ap.POL_EFF_DT DESC) = 1;
"""

# Get data from Snowflake
cs = ctx.cursor()
try:
    cs.execute(query_scoremart)
    df = cs.fetch_pandas_all()
finally:
    cs.close()
ctx.close()

df.shape

(228503, 74)

In [3]:
# calculate relativities
all_state_mean_lr_dict = df.groupby('STATE').agg(all_state_mean_lr_dict = ('EXPECTED_LOSS_RATIO', 'mean')).to_dict()
df["all_state_lr_mean"] = df["STATE"].map(all_state_mean_lr_dict['all_state_mean_lr_dict'])
df['ULM_eLRR'] = df['EXPECTED_LOSS_RATIO'] / df['all_state_lr_mean']

all_state_mean_ol_lr_dict = df.groupby('STATE').agg(all_state_mean_ol_lr_dict = ('OL_EXPECTED_LOSS_RATIO', 'mean')).to_dict()
df["all_state_ol_lr_mean"] = df["STATE"].map(all_state_mean_ol_lr_dict['all_state_mean_ol_lr_dict'])
df['ULM_OL_eLRR'] = df['OL_EXPECTED_LOSS_RATIO'] / df['all_state_ol_lr_mean']

In [4]:
def summarize_data(dataframe, by_vars, aggdict, writer, outsheet):
    
    # create deep copy
    df_tmp = copy.deepcopy(dataframe)        
            
    # summarize
    table = df_tmp.pivot_table(index = by_vars, aggfunc = aggdict).reset_index()
    
    # write to excel
    table.to_excel(writer, sheet_name=outsheet, index = False)
    # writer.save()

In [5]:
# aggregations
aggregation_dict = {'COMPL_QTE_IND': 'count',
           'ULM_eLRR': 'mean',
#           'ULM_OL_eLRR': 'mean',
           }

In [6]:
# Primary Named Insured age
df.loc[df['PNI_AGE'] < 50, 'PNI_AGE_grp'] = '<50'
df.loc[(df['PNI_AGE'] >=50) & (df['PNI_AGE'] <60), 'PNI_AGE_grp'] = '50-59'
df.loc[(df['PNI_AGE'] >=60) & (df['PNI_AGE'] <70), 'PNI_AGE_grp'] = '60-69'
df.loc[(df['PNI_AGE'] >=70) & (df['PNI_AGE'] <80), 'PNI_AGE_grp'] = '70-79'
df.loc[df['PNI_AGE'] >=80, 'PNI_AGE_grp'] = '80+'
df['PNI_AGE_grp'].value_counts()

PNI_AGE_grp
60-69    83822
70-79    63862
50-59    60075
80+      19677
<50       1067
Name: count, dtype: int64

In [7]:
# Insurance Score Deciles
df.loc[(df['CV_PLCY_CREDITVISION_CW'] ==0)   | (df['CV_PLCY_CREDITVISION_CW'] >= 998), 'cv_decile'] = 'NH/NS'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 0)   & (df['CV_PLCY_CREDITVISION_CW'] <= 504), 'cv_decile'] = 'Decile 10'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 504) & (df['CV_PLCY_CREDITVISION_CW'] <= 603), 'cv_decile'] = 'Decile 09'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 603) & (df['CV_PLCY_CREDITVISION_CW'] <= 670), 'cv_decile'] = 'Decile 08'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 670) & (df['CV_PLCY_CREDITVISION_CW'] <= 714), 'cv_decile'] = 'Decile 07'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 714) & (df['CV_PLCY_CREDITVISION_CW'] <= 744), 'cv_decile'] = 'Decile 06'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 744) & (df['CV_PLCY_CREDITVISION_CW'] <= 766), 'cv_decile'] = 'Decile 05'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 766) & (df['CV_PLCY_CREDITVISION_CW'] <= 785), 'cv_decile'] = 'Decile 04'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 785) & (df['CV_PLCY_CREDITVISION_CW'] <= 804), 'cv_decile'] = 'Decile 03'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 804) & (df['CV_PLCY_CREDITVISION_CW'] <= 825), 'cv_decile'] = 'Decile 02'
df.loc[(df['CV_PLCY_CREDITVISION_CW'] > 825) & (df['CV_PLCY_CREDITVISION_CW'] <  998), 'cv_decile'] = 'Decile 01'
df['cv_decile'].value_counts()

cv_decile
Decile 10    43065
Decile 09    27946
Decile 08    24201
Decile 07    22456
Decile 06    21107
Decile 05    19324
Decile 04    18467
Decile 03    17921
Decile 02    16074
Decile 01    13588
NH/NS         4354
Name: count, dtype: int64

In [8]:
# UBI Score
df.loc[df['TERM_LATST_UBI_SCORE'] < 10, 'ubi_score'] = '0-9'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 10) & (df['TERM_LATST_UBI_SCORE'] <= 19), 'ubi_score'] = '10-19'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 20) & (df['TERM_LATST_UBI_SCORE'] <= 29), 'ubi_score'] = '20-29'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 30) & (df['TERM_LATST_UBI_SCORE'] <= 39), 'ubi_score'] = '30-39'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 40) & (df['TERM_LATST_UBI_SCORE'] <= 49), 'ubi_score'] = '40-49'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 50) & (df['TERM_LATST_UBI_SCORE'] <= 59), 'ubi_score'] = '50-59'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 60) & (df['TERM_LATST_UBI_SCORE'] <= 69), 'ubi_score'] = '60-69'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 70) & (df['TERM_LATST_UBI_SCORE'] <= 79), 'ubi_score'] = '70-79'
df.loc[(df['TERM_LATST_UBI_SCORE'] >= 80) & (df['TERM_LATST_UBI_SCORE'] <= 89), 'ubi_score'] = '80-89'
df.loc[df['TERM_LATST_UBI_SCORE'] >= 90, 'ubi_score'] = '90+'
df['ubi_score'].value_counts()

ubi_score
0-9      159888
40-49      9847
30-39      9591
20-29      9437
10-19      8413
50-59      6673
60-69      6079
90+        5987
70-79      5536
80-89      5409
Name: count, dtype: int64

In [9]:
# Telematics Bucket Code
df['tlmtc_bucket_cd'] = 'Not Active'
df.loc[df['TLMTC_POL_BUCKET_CD'].isin(['Full Participation', 'Partial Participation', 'Enrollment Window']), 'tlmtc_bucket_cd'] = df['TLMTC_POL_BUCKET_CD']
df.loc[df['TLMTC_POL_BUCKET_CD'] == 'Issued with TrueLane but Never Activated', 'tlmtc_bucket_cd'] = 'Issued with TrueLane but not active'
df.loc[df['TLMTC_POL_BUCKET_CD'] == 'Issued with TrueLane, Activated,Valid Score, but Unenrolled', 'tlmtc_bucket_cd'] = 'Issued with TrueLane but not active'
df.loc[df['TLMTC_POL_BUCKET_CD'] == 'Issued with TrueLane, Activated,No Score, but Unenrolled', 'tlmtc_bucket_cd'] = 'Issued with TrueLane but not active'
df['tlmtc_bucket_cd'].value_counts()

tlmtc_bucket_cd
Full Participation                     96223
Issued with TrueLane but not active    95333
Partial Participation                  33975
Enrollment Window                       2855
Not Active                               117
Name: count, dtype: int64

In [10]:
writer = pd.ExcelWriter('./Q3_Auto_Segmentation.xlsx', engine = 'xlsxwriter')
summarize_data(dataframe = df, by_vars = ['PNI_AGE_grp', ], aggdict = aggregation_dict, writer = writer, outsheet = 'PNI_AGE_grp')
summarize_data(dataframe = df, by_vars = ['cv_decile', ], aggdict = aggregation_dict, writer = writer, outsheet = 'cv_decile')
summarize_data(dataframe = df, by_vars = ['ubi_score', ], aggdict = aggregation_dict, writer = writer, outsheet = 'ubi_score')
summarize_data(dataframe = df, by_vars = ['tlmtc_bucket_cd', ], aggdict = aggregation_dict, writer = writer, outsheet = 'tlmtc_bucket_cd')
writer.close()